# Python PIL (Pillow) benchmark
Since the Julia Benchmark package [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) automatically selects the number of samples, the Python benchmarks (computed using [timeit](https://github.com/python/cpython/blob/3.8/Lib/timeit.py)) are configured according to the number of samples BenchmarkTools takes.

Our tests consist of the following benchmarks:
1. Loading images
2. Saving images
3. Resizing (Scaling) images
4. Rotating Images
5. Getting the histogram (Grayscale Histogram)
6. Equalizing the Grayscale Histogram
7. Converting image to grayscale
8. Changing colorspace to HSV
9. Applying Gaussian Blur

In [2]:
import PIL
import timeit

## 1. Loading images

In [3]:
s = """\
img = Image.open("images/lighthouse.png")
"""

setup = "from PIL import Image"

print(str(round((timeit.timeit(stmt = s, setup = setup, number = 44)/44)*1000, 3)) + " ms")

3.758 ms


## 2. Saving images

In [4]:
s = """\
img.save("imagecopy.png")
"""

setup = """\
from PIL import Image
img = Image.open("images/lighthouse.png")
"""

print(str(round((timeit.timeit(stmt = s, setup = setup, number = 50)/50)*1000, 3)) + " ms")

309.918 ms


## 3. Resizing (Scaling) images

In [5]:
s = """\
xsize, ysize = img.size
img.resize((xsize*3, ysize*3))
"""

setup = """\
from PIL import Image
img = Image.open("images/lighthouse.png")
"""

print(str(round((timeit.timeit(stmt = s, setup = setup, number = 22)/22)*1000, 3)) + " ms")

19.11 ms


## 4. Rotating Images

In [6]:
s = """\
img.rotate(90)
"""

setup = """\
from PIL import Image
img = Image.open("images/lighthouse.png")
"""

print(str(round((timeit.timeit(stmt = s, setup = setup, number = 184)/184)*1000, 3)) + " ms")

3.649 ms


## 5. Getting the histogram (Grayscale Histogram)

In [7]:
s = """\
img.histogram()
"""

setup = """\
from PIL import Image
img = Image.open("images/lighthouse.png")
"""

print(str(round((timeit.timeit(stmt = s, setup = setup, number = 125)/125)*1000, 3)) + " ms")

2.143 ms


## 6. Equalizing the Grayscale Histogram

In [8]:
s = """\
PIL.ImageOps.equalize(img)
"""

setup = """\
import PIL
img = PIL.Image.open("images/lighthouse.png")
from PIL import ImageOps
"""

print(str(round((timeit.timeit(stmt = s, setup = setup, number = 63)/63)*1000, 3)) + " ms")

5.322 ms


## 7. Converting image to grayscale

In [9]:
s = """\
PIL.ImageOps.grayscale(img)
"""

setup = """\
import PIL
img = PIL.Image.open("images/lighthouse.png")
from PIL import ImageOps
"""

print(str(round((timeit.timeit(stmt = s, setup = setup, number = 1270)/1270)*1000, 3)) + " ms")

0.831 ms


## 8. Changing colorspace to HSV

In [10]:
s = """\
img.convert('HSV')
"""

setup = """\
import PIL
img = PIL.Image.open("images/lighthouse.png")
from PIL import ImageOps
"""

print(str(round((timeit.timeit(stmt = s, setup = setup, number = 362)/362)*1000, 3)) + " ms")

26.352 ms


## 9. Applying Gaussian Blur

In [11]:

s = """\
gauss = img.filter(PIL.ImageFilter.GaussianBlur(radius=3))
"""

setup = """\
import PIL
img = PIL.Image.open("images/lighthouse.png")
from PIL import ImageFilter
"""

print(str(round((timeit.timeit(stmt = s, setup = setup, number = 75)/75)*1000, 3)) + " ms")

33.943 ms


## Summary:

Since `Images.jl` gives us the mean time while `timeit` gives us the total time of the benchmark, we have taken the mean time in `timeit` so that the benchmarks are comparable.

### Table for comparison

Here is the table of the Benchmarks:

| Testcase/Framework               | Images.jl  | Pillow (PIL) | OpenCV    |
|----------------------------------|------------|--------------|-----------|
| Loading images                   | 114.129 ms | 3.758 ms     | 17.943 ms |
| Saving images                    | 100.002 ms | 309.918 ms   | 37.732 ms |
| Resizing (Scaling) images        | 230.759 ms | 19.11 ms     | 5.656 ms  |
| Rotating images                  | 27.162 ms  | 3.649 ms     | 2.88 ms   |
| Calculating Greyscale Histogram  | 39.977 ms  | 2.143 ms     | 0.418 ms  |
| Greyscale Histogram equalization | 80.208 ms  | 5.322 ms     | 0.008 ms  |
| Converting to Greyscale          | 3.931 ms   | 0.831 ms     | 0.212 ms  |
| Changing colorspace to HSV       | 13.802 ms  | 26.352 ms    | 1.277 ms  |
| Applying Gaussian Blur           | 67.063 ms  | 33.943 ms    | 1.626 ms  |

We find that Pillow (PIL) consistently gives sub great timings for the operations with the one outlier of saving images, while OpenCV varies wildly and takes comparatively (to itself) more time for some basic tasks related to inputting and outputting files.

Comparing Images.jl to the other 2 frameworks, Images.jl is never the fastest, which is a point where significant improvement can be made. However, Images.jl compares well with the two other frameworks when the operation includes changing the colorspace and saving images. Another point of concern is the fact that Images.jl is the slowest of the 3 frameworks in 7/9 of the tests. This might be due to OpenCV's powerful C and C++ bindings, or because of the powerful and fast `numpy` backend that Python has.

Maybe the algorithms for Images.jl are not the most efficient? Maybe these algorithms need updating?